## Normalization
- 정의 학습을 안정화하고 gradient 폭주 or 소실을 줄이고, 수렴 속도를 빠르게 하는 기법
- 정의, 효과, 장점, 단점, 사용 논문 예시, 시뮬레이션

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## 1. Batch Normalization(BatchNorm)
#### 1-1. 정의
- 등장 배경(gradient flow 문제)
  - 딥러닝에서 **gradient가 사라지거나 폭주**하는 문제
  - 딥러닝 모델이 학습할 때, 레이어 입력 값의 분포가 계속 변하는 문제(Internal Covariate Shift)를 줄여주는 정규화 기법
- 입력(mini batch) x를 평균 0, 분산 1로 정규화(각 feature의 scale을 일정하게 유지)
  - $\hat{x}_i=\frac{x_i-\mu_B}{\sqrt{\sigma^2_B + \epsilon}}, \quad y_i = \gamma\hat{x}_i+\beta$
  - $x_i$: 미니배치 입력
  - $\mu_B, \sigma^2_B$: 미니배치 평균과 분산
  - $\gamma, \beta$: 학습 가능한 scale & shift 파라미터
  - $\epsilon$: 안정화를 위한 작은 값

#### 1-2. 장점
- 학습 안정화: gradient exploding/vanishing 위험이 크게 감소
- 학습 속도 향상
  - 더 큰 lr 사용 가능
  - SGD가 훨씬 잘 작동
- 성능 향상: CNN, MLP에서 성능 향상 명확
- 일종의 regularization 효과
  - 각 배치마다 mean, var이 달라서 dropout 효과와 비슷
  - 과적합 방지
  
#### 1-3. 단점
- 배치 크기에 민감
  - Batch size가 작으면 mean, var이 불안정해짐
- 시퀀스 길이가 변하거나 온라인 학습에서는 부적합
  - RNN, Transformer 배치 단위 통계(Mean, Var)가 불안정하고 sequence length등 다름
  - LayerNorm
- 추론 때는 moving average 통계를 사용해야 해서 설정이 까다로움
- GPU 병렬성 제약
  - 분산 학습 시 각 GPU마다 batch 통계가 달라지는 문제
  - SyncBatchNorm 필요
  
#### 1-4. 사용예시
- 이미지 처리(CNN): ResNet, VGG, EfficientNet
- 대규모 FC 네트워크(MLP)
- GAN 일부

#### 1-5. 용어설명
- Covariate
  - 통계학에서 "변수들 간의 관계를 설명하는 독립 변수"
  - 쉽게 말하면 관찰된는 입력 변수
    - 예1. 키와 몸무게 관계를 연구한다면, 키가 covariate, 몸무게가 결과(y)
    - 예2. 딥러닝에서는 layer로 들어오는 입력 x가 covariate
- Internal Covariate
  - 네트워크 내부의 covariate
  - 즉, 각 layer로 들어가는 입력(feature) 자체가 학습 과정에서 계속 변하는 것
- 문제 상황
  - Layer1 → Layer2 → Layer3 순으로 학습할 때, Layer1이 학습되면서 출력이 조금씩 바뀌면
  - Layer2 입력 분포도 계속 변함 → graident 학습에 방해
  - 즉, 네트워크 내부 feature의 분포가 학습 도중에 계속 이동하는 현상
    - BatchNorm은 layer1의 output(z1)을 평균 0, 분산 1로 정규화 
    - layer2가 일정한 분포의 입력을 받아 안정적인 gradient flow

In [37]:
class MLP(nn.Module):
    def __init__(self, batchnorm=False):
        super().__init__()
        self.fc1 = nn.Linear(10, 64)
        self.bn1 = nn.BatchNorm1d(64) if batchnorm else nn.Identity()
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
        

# Dataset: y = 3x + noise
x = torch.randn(1000, 10)
y = x[:, 0] + torch.randn(1000) * 0.1
y = y.unsqueeze(1)

# Simple Comparison
no_bn_model = MLP(False)
no_bn_optimizer = torch.optim.SGD(no_bn_model.parameters(), lr=1e-3)

bn_model = MLP(True)
bn_optimizer = torch.optim.SGD(bn_model.parameters(), lr=1e-3)

criterion = nn.MSELoss()
for epoch in range(100):
    # no bn
    no_bn_optimizer.zero_grad()
    no_bn_pred = no_bn_model(x)
    no_bn_loss = criterion(no_bn_pred, y)
    no_bn_loss.backward()
    no_bn_optimizer.step()
    
    # bn
    bn_optimizer.zero_grad()
    bn_pred = bn_model(x)
    bn_loss = criterion(bn_pred, y)
    bn_loss.backward()
    bn_optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1} | DIFF: {no_bn_loss.item() - bn_loss.item():.4f} | No BN Loss: {no_bn_loss.item():.4f} | BN Loss: {bn_loss.item():.4f}")
        

print(f"MEAN:{bn_model.bn1.running_mean}")
print(f"VAR:{bn_model.bn1.running_var}")

Epoch 10 | DIFF: -0.1921 | No BN Loss: 1.1506 | BN Loss: 1.3427
Epoch 20 | DIFF: -0.0114 | No BN Loss: 1.1243 | BN Loss: 1.1357
Epoch 30 | DIFF: 0.1117 | No BN Loss: 1.0990 | BN Loss: 0.9873
Epoch 40 | DIFF: 0.1983 | No BN Loss: 1.0747 | BN Loss: 0.8764
Epoch 50 | DIFF: 0.2613 | No BN Loss: 1.0513 | BN Loss: 0.7900
Epoch 60 | DIFF: 0.3088 | No BN Loss: 1.0286 | BN Loss: 0.7199
Epoch 70 | DIFF: 0.3456 | No BN Loss: 1.0067 | BN Loss: 0.6611
Epoch 80 | DIFF: 0.3748 | No BN Loss: 0.9854 | BN Loss: 0.6106
Epoch 90 | DIFF: 0.3985 | No BN Loss: 0.9647 | BN Loss: 0.5662
Epoch 100 | DIFF: 0.4180 | No BN Loss: 0.9446 | BN Loss: 0.5266
MEAN:tensor([ 0.0635,  0.1785, -0.0881,  0.0043, -0.0157, -0.3289,  0.2877,  0.0256,
         0.0033, -0.1295,  0.1910,  0.2438,  0.1987, -0.0557,  0.1170, -0.2819,
        -0.2182, -0.2477,  0.2318,  0.1992,  0.2348, -0.1188,  0.0194,  0.2432,
         0.1643, -0.1981,  0.1550,  0.0160,  0.0167,  0.0288,  0.2727, -0.0576,
        -0.0360,  0.0059,  0.2292, -0.2853

## 2. Layer Normalization(LayerNorm)
#### 2-1. 정의
- 등장 배경(gradient flow 문제)
  - 딥러닝에서 **gradient가 사라지거나 폭주**하는 문제
  - 특히 **RNN, Transformer** 같은 Sequence 모델에서는 BatchNorm 적용이 어려움
    - 미니배치 통계(Mean, Var)가 Sequence 길이에 따라 달라지거나 batch size가 작은 경우가 많음
- **LayerNorm**은 배치 단위가 아니라 feature 단위로 정규화
  - 즉, 한 sample 내 모든 feature를 평균 0, 분산 1로 변환
  - $\hat{x}_i=\frac{x_i-\mu_L}{\sqrt{\sigma^2_L + \epsilon}}, \quad y_i = \gamma\hat{x}_i+\beta$
  - $x_i$: 한 샘플의 feature
  - $\mu_L, \sigma^2_L$: 한 샘플의 평균과 분산
  - $\gamma, \beta$: 학습 가능한 scale & shift 파라미터
  - $\epsilon$: 안정화를 위한 작은 값
    - **차이점**: BatchNorm은 batch dimension, LayerNorm은 feature dimension
    - batchnorm은 sample 내 feature가 독립적일 때 (sex, age, country, ...) 
    - layernorm은 sample 내 feature가 의존적일 때 sentence

#### 2-2. 장점
- **배치 크기 독립적: batch size가 1이어도 안정적**
- Sequence 모델에 적합(RNN, Transformer)
- Gradient 안정화: 한 샘플 내 feature 분포를 일정하게 유지(gradient exploding/vanishing 감소)
- 학습 속도 향상: 안정적인 gradient flow 덕분에 학습이 빠르고 수렴이 잘 됨
  
#### 2-3. 단점
- **공간 정보 손실 가능**: CNN에서 feature map 정규화 시 spatial 정보를 무시
- BatchNorm 대비 약간 느린 학습: batch 단위 정규화가 병렬 처리에 더 효율적
- 일부 연구에서는 regularization 효과가 BatchNorm보다 약함
  
#### 2-4. 사용예시
- Sequence Modeling: RNN, LSTM, GRU
- Attention: Transformer, BERT, GPT, ViT 등

In [36]:
class MLP(nn.Module):
    def __init__(self, layernorm=False):
        super().__init__()
        self.fc1 = nn.Linear(10, 64)
        self.ln1 = nn.LayerNorm(64) if layernorm else nn.Identity()
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.ln1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
        

# Dataset: y = 3x + noise
x = torch.randn(1000, 10)
y = x[:, 0] + torch.randn(1000) * 0.1
y = y.unsqueeze(1)

# Simple Comparison
no_ln_model = MLP(False)
no_ln_optimizer = torch.optim.SGD(no_ln_model.parameters(), lr=1e-3)

ln_model = MLP(True)
ln_optimizer = torch.optim.SGD(ln_model.parameters(), lr=1e-3)

criterion = nn.MSELoss()
for epoch in range(100):
    # no bn
    no_ln_optimizer.zero_grad()
    no_ln_pred = no_ln_model(x)
    no_ln_loss = criterion(no_ln_pred, y)
    no_ln_loss.backward()
    no_ln_optimizer.step()
    
    # bn
    ln_optimizer.zero_grad()
    ln_pred = ln_model(x)
    ln_loss = criterion(ln_pred, y)
    ln_loss.backward()
    ln_optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1} | DIFF: {no_ln_loss.item() - ln_loss.item():.4f} | No LN Loss: {no_ln_loss.item():.4f} | LN Loss: {ln_loss.item():.4f}")

print(f"WEIGHT:{ln_model.ln1.weight}")
print(f"BIAS:{ln_model.ln1.bias}")

Epoch 10 | DIFF: -0.1564 | No LN Loss: 0.9443 | LN Loss: 1.1007
Epoch 20 | DIFF: -0.1073 | No LN Loss: 0.9206 | LN Loss: 1.0279
Epoch 30 | DIFF: -0.0667 | No LN Loss: 0.8978 | LN Loss: 0.9645
Epoch 40 | DIFF: -0.0322 | No LN Loss: 0.8757 | LN Loss: 0.9079
Epoch 50 | DIFF: -0.0021 | No LN Loss: 0.8543 | LN Loss: 0.8563
Epoch 60 | DIFF: 0.0246 | No LN Loss: 0.8335 | LN Loss: 0.8089
Epoch 70 | DIFF: 0.0484 | No LN Loss: 0.8133 | LN Loss: 0.7649
Epoch 80 | DIFF: 0.0698 | No LN Loss: 0.7937 | LN Loss: 0.7239
Epoch 90 | DIFF: 0.0890 | No LN Loss: 0.7746 | LN Loss: 0.6856
Epoch 100 | DIFF: 0.1064 | No LN Loss: 0.7561 | LN Loss: 0.6497
WEIGHT:Parameter containing:
tensor([1.0040, 1.0032, 1.0005, 0.9999, 1.0000, 0.9975, 0.9996, 1.0005, 0.9974,
        0.9992, 1.0003, 0.9981, 0.9994, 1.0030, 0.9994, 1.0025, 0.9978, 0.9995,
        0.9991, 0.9965, 1.0007, 1.0004, 1.0024, 0.9992, 1.0008, 1.0002, 0.9993,
        0.9999, 0.9997, 0.9992, 0.9999, 1.0012, 0.9988, 1.0010, 0.9967, 1.0026,
        0.9999,

## 3. Instance Normalization(InstanceNorm)
#### 3-1. 정의
- 등장 배경(gradient flow 문제)
  - **Style Transfer, Image Generation** 에서 등장
  - CNN feature map마다 **style(밝기, 대비 등) 차이**가 심할 때, gradient flow 안정화 필요
- **InstanceNorm**은 배치 단위가 아니라 각 샘플의 각 채널(feature map)별로 정규화
  - 즉, batch-independent, channel-wise normalization
  - $\hat{x}_{n, c, h, w}=\frac{x_{n, c, h, w}-\mu_{n, c}}{\sqrt{\sigma^2_{n, c} + \epsilon}}, \quad y_{n, c, h, w} = \gamma\hat{x}_{n, c, h, w}+\beta_c$
  - $n$: 배치 index
  - $c$: channel
  - $\mu_{n,c}, \sigma^2_{n,c}:$: 한 샘플의 한 채널에 대한 평균/분산
  - $\gamma_c, \beta_c$: 채널별 학습 가능한 scale & shift 파라미터
    
#### 3-2. 장점
- **스타일/조명 불변성 확보**
  - style transfer에서 style에 관계없이 content를 잘 학습 가능
- 배치 크기 독립적(batch size가 1이어도 안정적 학습 가능)
- Gradient 안정화: CNN feature map이 지나면서 gradient exploding/vanishing 감소
  
#### 3-3. 단점
- BatchNorm 대비 일반화 효과 낮음(regularization 효과 없음)
- 내용과 style분리가 필요 없는 일반 학습에는 효과 없음
  - 시퀀스 모델에는 적합하지 않음
  
#### 3-4. 사용예시
- Image Style Transfer: AdaIN(Adative InstanceNorm) 등
- CNN 기반 이미지 생성 모델
- GANs: CycleGAN, StyleGAN 등


#### 3-5. 적용 예시
- x: (4, 3, 32, 32)
- self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
- self.in1 = nn.InstanceNorm2d(16)
  
```
Input: (4, 3, 32, 32)  -> Conv2d -> (4, 16, 32, 32)
                   -> InstanceNorm2d:
                      batch 0, channel 0 -> 평균 0, 분산 1
                      batch 0, channel 1 -> 평균 0, 분산 1
                      ...
                      batch 3, channel 15 -> 평균 0, 분산 1
Output: (4, 16, 32, 32) 정규화된 feature map
```

- 배치끼리 통계 공유하지 않음(batch size 1도 안정적)
- 채널 단위로 정규화(스타일 제거, content 정보 보존)

In [30]:
class CNN(nn.Module):
    def __init__(self, instancenorm=False):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.in1 = nn.InstanceNorm2d(16) if instancenorm else nn.Identity()
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.in1(x)
        # 각 샘플별, 각 채널별 평균과 분산 계산
        # batch[0], channel[0] > 32x32 값의 평균0_0, 분산0_0
        # batch[0], channel[1] > 32x32 값의 평균0_1, 분산0_1
        x = self.relu(x)
        x = self.conv2(x)
        return x

# Dummy input: batch 4, RGB image 32x32
x = torch.randn(4, 3, 32, 32)

model = CNN(instancenorm=True)
out = model(x)
print(out.shape)  # (4, 32, 32, 32)

for p in model.in1.parameters():
    print(p)

torch.Size([4, 32, 32, 32])


## 4. Group Normalization(GroupNorm)
#### 4-1. 정의
- 등장 배경
  - BatchNorm의 단점: 작은 Batch Sized에서 불안정
  - BatchNorm 없이도 gradient flow 안정화 가능하도록 고안
- 핵심 아이디어
  - 채널을 여러 그룹으로 나누고, 각 그룹 내에서 정규화
  - 즉, BatchNorm과 InstanceNorm의 중간 형태
  - $\hat{x}_{n, g}=\frac{x_{n, g}-\mu_{n,g}}{\sqrt{\sigma^2_{n,g} + \epsilon}}, \quad y_{n, g} = \gamma_g\hat{x}_{n,g}+\beta_g$
  - $n$: 샘플 인덱스
  - $g$: 그룹 인덱스(각 그룹 내 채널 정규화)
  - $\mu_{n,g}, \sigma^2_{n, g}$: 그룹 내 평균, 분산
  - $\gamma_g, \beta_g$: 학습 가능한 scale & shift 파라미터
  - $\epsilon$: 안정화를 위한 작은 값
    - 특징:
    - InstanceNorm = 채널 1개씩 그룹화
    - LayerNorm = 전체 채널을 1개의 그룹

#### 4-2. 장점
- **배치 크기 독립적: batch size가 1이어도 안정적**
- Gradient 안정화: gradient exploding/vanishing 감소
- CNN에서 공간 정보 보존 가능
- BatchNorm보다 작은 배치, GPU memory 제약 환경에 강함
  
#### 4-3. 단점
- 채널 그룹 수 하이퍼파라미터 필요
  - 그룹 개수(G)가 너무 크거나 작으면 성능에 영향
- BatchNorm의 regularization 효과는 일부 감소
  
#### 4-4. 사용예시
- Small-batch CNN: object detection, segmentation 등
- ResNeXT, Detectron2 일부 모듈
- BatchNorm 적용이 어려움 medical image segmentation에서도 사용

#### 4-5. gradient flow 관점
- 그룹 단위 정규화 → channel correlation 유지하면서도 gradient 안정화
- BatchNorm처럼 batch 통계 의존 X → 작은 batch size에서도 학습 안정적
- InstanceNorm처럼 style 제거보다는 content 정보 보존


In [15]:
class CNN(nn.Module):
    def __init__(self, groupnorm=False, G=4):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.gn1 = nn.GroupNorm(num_groups=G, num_channels=16) if groupnorm else nn.Identity()
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.gn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        return x

# Dummy input: batch 4, RGB 32x32
x = torch.randn(4, 3, 32, 32)
model = CNN(groupnorm=True, G=4)
out = model(x)
print(out.shape)  # (4, 32, 32, 32)

torch.Size([4, 32, 32, 32])


In [19]:
for p in model.gn1.parameters():
    print(p)

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)


In [30]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing

    def forward(self, pred, target):
        """
        Args:
            pred(logits): (batch_size, num_classes)
            target(class indices): (batch_size)
        """

        num_classes = pred.size(1)
        # one-hot vector
        with torch.no_grad():
            true_dist = torch.zeros_like(pred) # (batch_size, num_classes)
            # 0보다 조금 큰 값
            true_dist.fill_(self.smoothing / (num_classes - 1)) 
            # true_dist에 target index 부분의 값을 1-Smoothing으로 체우는 함수
            true_dist.scatter_(1, target.data.unsqueeze(1), 1.0 - self.smoothing)

        log_prob = F.log_softmax(pred, dim=1)
        loss = torch.mean(torch.sum(-true_dist * log_prob, dim=1))
        return loss

In [31]:
batch_size = 10000
num_classes = 100

ce = nn.CrossEntropyLoss() # 기존
ce_smooth = LabelSmoothingCrossEntropy(smoothing=0.1)

pred = torch.randn(batch_size, num_classes, requires_grad=True)
target = torch.randint(low=0, high=num_classes-1, size=(batch_size,))

ce_loss = ce(pred, target)
ce_smooth_loss = ce_smooth(pred, target)


print(f"Cross Entropy Loss:{round(ce_loss.item(), 4)}")
print(f"Label Smooth Cross Entropy Loss:{round(ce_smooth_loss.item(), 4)}")

Cross Entropy Loss:5.0733
Label Smooth Cross Entropy Loss:5.0756
